In [1]:
import graphlab

#Load house sales data

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to prashantbhat94@gmail.com and will expire on February 13, 2020.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\user\AppData\Local\Temp\graphlab_server_1550998248.log.0


#Split the data into train and test data

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

In order to make use of the closed form solution as well as take advantage of graphlab's built in functions we will review some important ones. In particular:
* Computing the sum of an SArray
* Computing the arithmetic average (mean) of an SArray
* multiplying SArrays by constants
* multiplying SArrays by other SArrays

In [4]:
#Compute the mean of the House Price of the King County in 2 ways:

prices = sales['price']

sum_prices = prices.sum()
num_houses = prices.size()
avg_price1 = sum_prices/num_houses
avg_price2 = prices.mean()
print('Average by method 1:'+ str(avg_price1))
print('Average by method 2:'+str(avg_price2))

Average by method 1:540088.141905
Average by method 2:540088.141905


In [5]:
#Multiply every price by 0.5
half_prices  = 0.5 *prices

#Compute the sum of squares of prices
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum()
print("Sum of squared prices:"+str(sum_prices_squared))

Sum of squared prices:9.21732513355e+15


#Build a simple linear regression function

In [6]:
def simple_linear_regression(input_feature,output):
    Xi = input_feature
    Yi = output
    N = len(Xi)
    #compute the sum of the input feature and the ouptut
    Xi_sum = Xi.sum()
    Yi_sum = Yi.sum()
    
    #Compute the mean of the input feature and the output
    XMean = Xi.mean()
    YMean = Yi.mean()
    
    #Compute the product of the input and the output feature and its sum
    SumYiXi = (Yi*Xi).sum()
    YiXiN = ((Yi.sum()*Xi.sum()))/N
    
    #Compute the squared value of the input feature and the mean
    XiSq = (Xi*Xi).sum()
    XiXiN = (Xi.sum()*Xi.sum())/N
    
    #Use the formula for the slope
    slope = (SumYiXi - YiXiN)/(XiSq - XiXiN)
    
    #Use the formula for intercept
    intercept  = YMean - (slope*XMean)
    
    return(intercept,slope)
    

In [17]:
test_feature = graphlab.SArray(range(5))
test_output = graphlab.SArray(1+1*test_feature)
(test_intercept,test_slope)=simple_linear_regression(test_feature,test_output)
print('Intercept:'+str(test_intercept))
print('SLope:'+str(test_slope))

Intercept:1.0
SLope:1


In [8]:
sqft_intercept,sqft_slope = simple_linear_regression(train_data['sqft_living'],train_data['price'])
print('Intercept:'+str(sqft_intercept))
print('Slope:'+str(sqft_slope))      

Intercept:-47116.0765749
Slope:281.958838568


#Predicting Values

In [9]:
def get_regression_predictions(input_feature,intercept,slope):
    #Calculate the predicted values
    predicted_values = intercept + slope * input_feature
    return predicted_values

#Now we have build our own prediction function use that to answer the quiz questions
**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [10]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print ("The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price))

The estimated price for a house with 2650 squarefeet is $700074.85


**#Residual Sum Of Squares#**

Now the we have written a model and make a predictions let's evauate our model using Residual Sum Of Squares(RSS). Now write a function foe getting the RSS.

In [19]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    # First get the predictions
    predicted_values = intercept + (slope * input_feature)
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = output - predicted_values
    # square the residuals and add them up
    RSS = (residuals * residuals).sum()
    return(RSS)

In [20]:
print(get_residual_sum_of_squares(test_feature,test_output,test_intercept,test_slope))

0.0


Now use the function to calculate RSS on the training data.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [22]:
rss_prices_on_sq_ft = get_residual_sum_of_squares(train_data['sqft_living'],train_data['price'],sqft_intercept,sqft_slope)
print('The RSS of Prediting the prices based on the sqft is:'+ str(rss_prices_on_sq_ft))

The RSS of Prediting the prices based on the sqft is:1.20191835632e+15


# Predict the squarefeet given price

##What if we want to predict the square feet of the house given the price , inverse the prediction formula and let us see the output

In [25]:
def inverse_regression_predictions(output, intercept, slope):
    estimated_feature = (output - intercept)/slope
    return (estimated_feature)

##Now that we have a function to compute the square foot given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be

**Quiz Question: According to this function and the regression slope and the intercept from (3) what is the extimated square feet for a house costing $800,000 ?**

In [26]:
my_house_price = 800000
estimated_square_feet = inverse_regression_predictions(my_house_price,sqft_intercept,sqft_slope)
print("The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price,estimated_square_feet))

The estimated squarefeet for a house worth $800000.00 is 3004


# New Model: estimate price from bedrooms

Now we estimate the house price using the parameters bedrooms

In [28]:
#Estimate the slope and intercept for predicting the 'price' based on 'bedrooms'
bed_intercept,bed_slope = simple_linear_regression(train_data['bedrooms'],train_data['price'])

print("Intercept : "+str(bed_intercept))
print("SLope :" +str(bed_slope))

Intercept : 109473.180469
SLope :127588.952175


# Test your linear regression algorithm

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS and Test data?**

In [29]:
#Compute RSS when using bedrooms on test data
bed_intercept1,bed_slope1 = simple_linear_regression(train_data['bedrooms'],train_data['price'])

res_price_on_bedrooms = get_residual_sum_of_squares(test_data['bedrooms'],test_data['price'],bed_intercept1,bed_slope1)
print ("The RSS for prediting the house prices based on Bedrooms:" +str(res_price_on_bedrooms))

The RSS for prediting the house prices based on Bedrooms:4.93364582868e+14


In [30]:
#Compute the RSS when using sqft on test_data
sqft_intercept1,sqft_slope1= simple_linear_regression(train_data['sqft_living'],train_data['price'])

res_price_on_sqft = get_residual_sum_of_squares(test_data['sqft_living'],test_data['price'],sqft_intercept1,sqft_slope1)
print ("The RSS for predicting the house price based on Sqft Living: "+ str(res_price_on_sqft))

The RSS for predicting the house price based on Sqft Living: 2.75402936247e+14
